# Tesla stock price prediction using Elon Musk Tweets with Multinomial Naive Bayes

### Implementing the same algorithms as the paper (https://link.springer.com/chapter/10.1007/978-981-19-5090-2_19)

Load the dataset and split weekly Tweets into their own columns

In [5]:
import pandas as pd
import numpy as np
import re

dataset = pd.read_csv("results.csv")
tweets = dataset['Tweets'].tolist()
tweets_data = pd.DataFrame(tweets)
tweets_data = tweets_data[0].str.split(", '",len(tweets),expand=True)


tweets_data.set_axis(['Tweet ' + str(index + 1) for index in range(101)],axis=1,inplace=True)  
dataset = pd.concat([dataset,tweets_data],axis=1,join='inner')
dataset.drop(dataset.columns[0],axis=1,inplace=True)
dataset.drop(['Likes','Tweets'],axis=1,inplace=True)
dataset.fillna(value=np.nan,inplace=True)
dataset.head()

C:\Users\chaud\AppData\Local\Temp\ipykernel_14984\985602157.py:8: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  tweets_data = tweets_data[0].str.split(", '",len(tweets),expand=True)
C:\Users\chaud\AppData\Local\Temp\ipykernel_14984\985602157.py:11: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  tweets_data.set_axis(['Tweet ' + str(index + 1) for index in range(101)],axis=1,inplace=True)


,Close,Dif,Retweets,Tweet 1,Tweet 2,Tweet 3,Tweet 4,Tweet 5,Tweet 6,Tweet 7,...,Tweet 92,Tweet 93,Tweet 94,Tweet 95,Tweet 96,Tweet 97,Tweet 98,Tweet 99,Tweet 100,Tweet 101
0,13.384667,-0.745333,"[16, 254, 260, 1434, 79]","['@Lockyep Not allowed, according to HK regula...",We need to do one more minor rev on 8.0 and th...,Writing post now with details. Will publish on...,Major improvements to Autopilot coming with V8...,@newscientist uh oh'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13.185333,-0.199334,"[368, 4887]",['Finishing Autopilot blog postponed to end o...,Loss of Falcon vehicle today during propellant...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13.220000,0.034667,"[851, 1406, 23, 247, 93, 89, 683, 840, 1223, 2...",['Will do some press Q&amp;A on Autopilot post...,Thoughtful Op-ed in Space News much appreciate...,Will get back to Autopilot update blog tomorrow.',@ashwin7002 @NASA @faa @AFPAA We have not rule...,Particularly trying to understand the quieter ...,"Support &amp; advice from @NASA, @FAA, @AFPAA ...",Important to note that this happened during a ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13.094000,-0.126000,"[7105, 4601]",['Scientists: Earth Endangered by New Strain o...,Climate change explained in comic book form by...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13.361333,0.267333,"[1083, 1242]","['Tesla Model S loses 28% after 50k miles, a M...",Comprehensive study by Autolist shows that a T...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Give each week a Sentiment based on if it resulted in the a increase or decrease in stock

In [6]:
#def giveSentimentRanges(cell):
    #if (-147.75 <= cell <= -79.31):
      #  return 0
    #elif(-79.30 <= cell <= -10.86):
       # return 1
    #elif(-10.85 <= cell <= 57.59):
        #return 2
    #elif(57.60 <= cell <= 126.04):
       # return 3
    #elif(126.05 <= cell <= 194.49):
      #  return 4 

def giveSentiment(cell):
    if (cell <= 0):
        return 0
    else:
        return 1

sentiment = list()

for diff in dataset['Dif']:
    sentiment.append(giveSentiment(diff))

dataset['Sentiment'] = sentiment
dataset.head()

C:\Users\chaud\AppData\Local\Temp\ipykernel_14984\190792637.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset['Sentiment'] = sentiment


,Close,Dif,Retweets,Tweet 1,Tweet 2,Tweet 3,Tweet 4,Tweet 5,Tweet 6,Tweet 7,...,Tweet 93,Tweet 94,Tweet 95,Tweet 96,Tweet 97,Tweet 98,Tweet 99,Tweet 100,Tweet 101,Sentiment
0,13.384667,-0.745333,"[16, 254, 260, 1434, 79]","['@Lockyep Not allowed, according to HK regula...",We need to do one more minor rev on 8.0 and th...,Writing post now with details. Will publish on...,Major improvements to Autopilot coming with V8...,@newscientist uh oh'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,13.185333,-0.199334,"[368, 4887]",['Finishing Autopilot blog postponed to end o...,Loss of Falcon vehicle today during propellant...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,13.220000,0.034667,"[851, 1406, 23, 247, 93, 89, 683, 840, 1223, 2...",['Will do some press Q&amp;A on Autopilot post...,Thoughtful Op-ed in Space News much appreciate...,Will get back to Autopilot update blog tomorrow.',@ashwin7002 @NASA @faa @AFPAA We have not rule...,Particularly trying to understand the quieter ...,"Support &amp; advice from @NASA, @FAA, @AFPAA ...",Important to note that this happened during a ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,13.094000,-0.126000,"[7105, 4601]",['Scientists: Earth Endangered by New Strain o...,Climate change explained in comic book form by...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,13.361333,0.267333,"[1083, 1242]","['Tesla Model S loses 28% after 50k miles, a M...",Comprehensive study by Autolist shows that a T...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Cleanup the data by removing unnecessary characters from the tweets

In [7]:
for column in dataset.columns[3:-1]:
    dataset[column] = dataset[column].str.lower()
    dataset[column] = dataset[column].replace('((\\n\()?\@\S+)','',regex=True)
    dataset[column] = dataset[column].replace('((\\n)?https\S+)','',regex=True)
    dataset[column] = dataset[column].replace('(\[)','',regex=True)
    dataset[column] = dataset[column].replace('(\])','',regex=True)
    dataset[column] = dataset[column].replace('(\&\S+)','',regex=True)
    dataset[column] = dataset[column].replace('(\#)','',regex=True)
    dataset[column] = dataset[column].replace('(\_)','',regex=True)
    dataset[column] = dataset[column].str.replace('([^\w\s#@/:%.,_-])','',regex=True,flags=re.UNICODE)
    dataset[column] = dataset[column].replace('([^\x00-\x7FâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]+)','',regex=True)
    dataset[column] = dataset[column].replace('(\d+)','',regex=True)
    dataset[column] = dataset[column].str.lstrip()
dataset = dataset.mask(dataset == '')
dataset.head()


,Close,Dif,Retweets,Tweet 1,Tweet 2,Tweet 3,Tweet 4,Tweet 5,Tweet 6,Tweet 7,...,Tweet 93,Tweet 94,Tweet 95,Tweet 96,Tweet 97,Tweet 98,Tweet 99,Tweet 100,Tweet 101,Sentiment
0,13.384667,-0.745333,"[16, 254, 260, 1434, 79]","not allowed, according to hk regulations. happ...",we need to do one more minor rev on . and then...,writing post now with details. will publish on...,major improvements to autopilot coming with v....,uh oh,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,13.185333,-0.199334,"[368, 4887]",finishing autopilot blog postponed to end of ...,loss of falcon vehicle today during propellant...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,13.220000,0.034667,"[851, 1406, 23, 247, 93, 89, 683, 840, 1223, 2...",will do some press q on autopilot post at am p...,thoughtful op-ed in space news much appreciate...,will get back to autopilot update blog tomorrow.,"we have not ruled that out., nope, it wasnt me",particularly trying to understand the quieter ...,support advice from others much appreciat...,important to note that this happened during a ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,13.094000,-0.126000,"[7105, 4601]",scientists: earth endangered by new strain of ...,climate change explained in comic book form by...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,13.361333,0.267333,"[1083, 1242]","tesla model s loses % after k miles, a mercede...",comprehensive study by autolist shows that a t...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Stemming and Lemmatization of the tweets

Note: you must uncomment the "nltk.download" lines the first time you run the code

In [8]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk


#Note- YOU WILL NEED TO UNCOMMENT THESE 3 LINES FOR THE CODE TO WORK
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

stem = PorterStemmer()
lemma = WordNetLemmatizer()

def stemSentence(sentence):
    words=word_tokenize(sentence)
    stem_sentence=list()
    for word in words:
        stem_sentence.append(stem.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)


def LemmatizerSentence(sentence):
    words=word_tokenize(sentence)
    lemma_sentence=list()
    for word in words:
        lemma_sentence.append(lemma.lemmatize(word))
        lemma_sentence.append(" ")
    return "".join(lemma_sentence)

for column in dataset.columns[3:-1]:
    result = list()
    for row in dataset[column]: 
        if(type(row) != float):
            result.append(LemmatizerSentence(stemSentence(row)))

    dataset[column].loc[len(dataset[column])] = result
    
dataset.head()

,Close,Dif,Retweets,Tweet 1,Tweet 2,Tweet 3,Tweet 4,Tweet 5,Tweet 6,Tweet 7,...,Tweet 93,Tweet 94,Tweet 95,Tweet 96,Tweet 97,Tweet 98,Tweet 99,Tweet 100,Tweet 101,Sentiment
0,13.384667,-0.745333,"[16, 254, 260, 1434, 79]","not allowed, according to hk regulations. happ...",we need to do one more minor rev on . and then...,writing post now with details. will publish on...,major improvements to autopilot coming with v....,uh oh,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,13.185333,-0.199334,"[368, 4887]",finishing autopilot blog postponed to end of ...,loss of falcon vehicle today during propellant...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,13.220000,0.034667,"[851, 1406, 23, 247, 93, 89, 683, 840, 1223, 2...",will do some press q on autopilot post at am p...,thoughtful op-ed in space news much appreciate...,will get back to autopilot update blog tomorrow.,"we have not ruled that out., nope, it wasnt me",particularly trying to understand the quieter ...,support advice from others much appreciat...,important to note that this happened during a ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,13.094000,-0.126000,"[7105, 4601]",scientists: earth endangered by new strain of ...,climate change explained in comic book form by...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,13.361333,0.267333,"[1083, 1242]","tesla model s loses % after k miles, a mercede...",comprehensive study by autolist shows that a t...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Count-Vectorization of the tweets to convert them to numerical arrays that can be used by the model

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

each_row = dict()
all_tweets = ''
for row in dataset.index:
    row_tweets = ''
    for column in dataset.columns[3:-1]:
        if(type(dataset[column][row]) is str):
            row_tweets += dataset[column][row] + ' '
            all_tweets += dataset[column][row] + ' '
    each_row[row] = [row_tweets]

count_vector = list()
vectorizer.fit([all_tweets])
for keys in each_row:
    vector = vectorizer.transform(each_row[keys])
    count_vector.append(vector.toarray())

count_vector = np.reshape(count_vector,(np.shape(count_vector)[0],np.shape(count_vector)[2]))
print(np.shape(count_vector))

(1114, 12017)


Dataset to be used as Features and Target

In [10]:

#Word_occurances = pd.DataFrame()

Word_occurances= pd.DataFrame(count_vector, columns=vectorizer.get_feature_names())
Word_occurances['Sentiment'] = dataset['Sentiment']

Word_occurances.head()

C:\Users\chaud\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaargh,aargh,abandoned,abandoning,abdomen,aber,abide,ability,ablate,ablative,...,zone,zones,zooming,zooms,zootopia,zorbix,zork,zu,zx,Sentiment
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Splitting data into train and test + training model

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

words = Word_occurances.iloc[:,:-1]
X_train,X_test,y_train,y_test = train_test_split(words,Word_occurances['Sentiment'],test_size=0.2,random_state=4)

MNB = MultinomialNB()
MNB.fit(X_train,y_train)

prediction = MNB.predict(X_test)



Scores for the model

In [12]:
from sklearn import metrics


print("Accuracy: ",metrics.accuracy_score(y_test,prediction))
print("Precision: ",metrics.precision_score(y_test,prediction))
print("F1 Score: ",metrics.f1_score(y_test,prediction))
print("Confusion Matrix: \n", metrics.confusion_matrix(y_test,prediction))
print("\n\n Classification Report: \n", metrics.classification_report(y_test,prediction))

Accuracy:  0.5650224215246636
Precision:  0.5899280575539568
F1 Score:  0.6283524904214559
Confusion Matrix: 
 [[44 57]
 [40 82]]


 Classification Report: 
               precision    recall  f1-score   support

           0       0.52      0.44      0.48       101
           1       0.59      0.67      0.63       122

    accuracy                           0.57       223
   macro avg       0.56      0.55      0.55       223
weighted avg       0.56      0.57      0.56       223



Modification applied where the setiment scores given are now based on mean and deviation

In [13]:
from statistics import stdev

def giveSentimentRanges(cell):
    if (cell < mean - deviation):
        return 0
    elif(mean - deviation <= cell < mean):
        return 1
    elif(mean <= cell < mean + (deviation)):
        return 2
    elif(mean + (deviation) <= cell < mean + (deviation*2)):
        return 3
    elif(cell > mean + (deviation*2)):
        return 4 

deviation = stdev(dataset['Dif'])
mean = dataset['Dif'].mean()
range_sentiment = list()
for diff in dataset['Dif']:
    range_sentiment.append(giveSentimentRanges(diff))

Word_occurances['Sentiment_Ranged'] = range_sentiment
Word_occurances.head()


,aaargh,aargh,abandoned,abandoning,abdomen,aber,abide,ability,ablate,ablative,...,zones,zooming,zooms,zootopia,zorbix,zork,zu,zx,Sentiment,Sentiment_Ranged
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2


Scores for the modification

In [14]:
X_train,X_test,y_train,y_test = train_test_split(words,Word_occurances['Sentiment_Ranged'],test_size=0.2,random_state=4)

MNB_ranges = MultinomialNB()
MNB_ranges.fit(X_train,y_train)

ranges_prediction = MNB_ranges.predict(X_test)
#print(y_test)
print("Accuracy: ",metrics.accuracy_score(y_test,ranges_prediction))
print("Precision: ",metrics.precision_score(y_test,ranges_prediction,average='weighted'))
print("F1 Score: ",metrics.f1_score(y_test,ranges_prediction,average='weighted'))
print("Confusion Matrix: \n", metrics.confusion_matrix(y_test,ranges_prediction))
print("\n\n Classification Report: \n", metrics.classification_report(y_test,ranges_prediction))

Accuracy:  0.4125560538116592
Precision:  0.3414395768655858
F1 Score:  0.36214116733166196
Confusion Matrix: 
 [[ 0 16  1  0  0]
 [ 2 65 31  0  0]
 [ 0 58 27  0  0]
 [ 0  9  4  0  0]
 [ 0  8  2  0  0]]


 Classification Report: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.42      0.66      0.51        98
           2       0.42      0.32      0.36        85
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00        10

    accuracy                           0.41       223
   macro avg       0.17      0.20      0.17       223
weighted avg       0.34      0.41      0.36       223



C:\Users\chaud\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chaud\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chaud\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMe